<a href="https://colab.research.google.com/github/kvin-alxndr/GPT-2-ChatBot/blob/main/GPT-2-ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-2 Based Empathetic ChatBot


# Instalación de dependencias

In [ ]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00


# Importar bibliotecas y cargar conjuntos de datos


In [ ]:
import pandas as pd
from datasets import list_datasets, load_dataset

# Obtener la lista de todos los conjuntos de datos disponibles
all_datasets = list_datasets()

# Imprimir la cantidad de conjuntos de datos disponibles
num_datasets = len(all_datasets)
print(f"Número de conjuntos de datos disponibles: {num_datasets}")

# Verificar si 'empathetic_dialogues' está en la lista de conjuntos de datos
is_em_dialogue_available = 'empathetic_dialogues' in all_datasets
print(f"¿'empathetic_dialogues' disponible? {is_em_dialogue_available}")

# Cargar el conjunto de datos 'empathetic_dialogues'
em_dialogue = load_dataset('empathetic_dialogues')
em_dialogue_pandas = em_dialogue
em_dialogue_pandas.set_format(type='pandas')
df = em_dialogue_pandas['train'][:]
df.head()

<ipython-input-2-f821b166e197>:5: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  all_datasets = list_datasets()


Número de conjuntos de datos disponibles: 82108
¿'empathetic_dialogues' disponible? True


Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,


# Estructura del conjunto de datos

In [ ]:
# Imprimir la longitud del conjunto de entrenamiento
print(f"Longitud del conjunto de entrenamiento: {len(em_dialogue['train'])}")

# Obtener las expresiones del conjunto de entrenamiento
utterances_train = em_dialogue['train']['utterance']
print(utterances_train.head(15))

Longitud del conjunto de entrenamiento: 76673
0     I remember going to see the fireworks with my ...
1     Was this a friend you were in love with_comma_...
2                   This was a best friend. I miss her.
3                                   Where has she gone?
4                                    We no longer talk.
5     Oh was this something that happened because of...
6      it feels like hitting to blank wall when i se...
7                         Oh ya? I don't really see how
8                      dont you feel so.. its a wonder 
9     I do actually hit blank walls a lot of times b...
10     i virtually thought so.. and i used to get sw...
11                              Wait what are sweatings
12                           Hi how are you doing today
13                           doing good.. how about you
14    Im good_comma_ trying to understand how someon...
Name: utterance, dtype: object


# FINE TUNING

## Preprocesamiento de datos y creación de la clase del conjunto de datos

In [ ]:
from torch.utils.data import Dataset

class BotDataset(Dataset):
      def __init__(self, dataset,tokenizer):
        self.data = dataset

        self.utterances=[]
        for i in self.data['train']:
            for j in i['utterance']:
              self.utterances.append(j)

      #<bos> = inicio de cadena
      #<eos> = fin de cadena
        for idx, i in enumerate(self.utterances):
          try:
              self.utterances[idx] ="<bos> "+i+" <bot>: "+self.utterances[idx+1]+" <eos>"
          except:
              break

        self.utterances = self.utterances[:6800]
        print(self.utterances[0])


        self.utterances_encoded = tokenizer(self.utterances,max_length=50,truncation=True,padding="max_length",return_tensors="pt")
        self.input_ids = self.utterances_encoded['input_ids']
        self.attention_mask = self.utterances_encoded['attention_mask']

      def __len__(self):
        return len(self.utterances)


      def __getitem__(self,idx):
        return(self.input_ids[idx], self.attention_mask[idx])

## Configuración del modelo y el optimizador


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>", "bos_token": "<bos>", "eos_token": "<eos>"})
tokenizer.add_tokens(["<bot>:"])
model = GPT2LMHeadModel.from_pretrained("gpt2") #se cargan los pesos del modelo
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)
optimizer = Adam(model.parameters(), lr=1e-3)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Función de inferencia

In [ ]:
def generate_inference(inp):
    inp = f"<bos> {inp} <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a)
    output = tokenizer.decode(output[0],skip_special_tokens=True)
    return output

## Entrenamiento del modelo





In [ ]:
import tqdm
def train_model(bot_dataset, model, optimizer):
    epochs = 12
    for epoch in tqdm.tqdm(range(epochs)):
        for input_ids, attention_mask in bot_dataset:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            optimizer.zero_grad()
            loss = model(input_ids, attention_mask=attention_mask, labels=input_ids).loss
            loss.backward()
            optimizer.step()

        torch.save(model.state_dict(), "chatbot.pt")
        print(generate_inference("hello, how are you"))

## Crear el conjunto de datos y DataLoader

In [ ]:
from torch.utils.data import DataLoader

Bot_dataset = BotDataset(em_dialogue, tokenizer)
Bot_dataloader = DataLoader(Bot_dataset, batch_size=64)
model.train()

# Entrenar el modelo
train_model(Bot_dataloader, model, optimizer)

<bos> Yeah about 10 years ago I had a horrifying experience. It was 100% their fault but they hit the water barrels and survived. They had no injuries but they almost ran me off the road. <bot>: Did you suffer any injuries? <eos>


  0%|          | 0/12 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  8%|▊         | 1/12 [01:37<17:53, 97.57s/it]

 hello, how are you  <bot>:   I am going to Daytona and Orlando for 10 days.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 17%|█▋        | 2/12 [03:15<16:15, 97.60s/it]

 hello, how are you  <bot>:  


...

 hello, how are you  <bot>:  


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 58%|█████▊    | 7/12 [11:23<08:07, 97.53s/it]

 hello, how are you  <bot>:  


 58%|█████▊    | 7/12 [12:40<09:03, 108.67s/it]


KeyboardInterrupt: ignored

## Prueba rapida

In [ ]:
print(generate_inference("i feel sad because my dog got lost"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 i feel sad because my dog got lost  <bot>:   I'm sorry to hear that. 


# Descarga de modelo

In [ ]:
from google.colab import files

files.download("chatbot.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Prueba GPT-2


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2") #se cargan los pesos del modelo
model = model.to(device)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
test_out = model.generate(**tokenizer("what is your greatest dream?",return_tensors="pt").to(device))[0]
print(tokenizer.decode(test_out,skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


what is your greatest dream?

I'm going to be a professional wrestler. I'm going


In [ ]:
def print_prompt_test(nombre_archivo):
    try:
        with open(nombre_archivo, 'r') as archivo:
            # Lee las líneas del archivo y las almacena en una lista
            lineas = archivo.read().splitlines()
    except FileNotFoundError:
        print(f"El archivo '{nombre_archivo}' no fue encontrado.")
        return None
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo: {e}")
        return None
    for cadena in lineas:
      out = generate_inference(cadena)
      print(out)

print_prompt_test('/content/drive/MyDrive/PLN/PROYECTO/Prompts_prueba.txt')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<bos> I feel sad because my dog got lost <bot>: ༼ つ �
<bos> I am feeling good today    <bot>:  I am feeling good
<bos> I am having a really hard time and i am feeling sad <bot>: ive
<bos> Today it's a very special day for my <bot>: ~~~ <bos


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<bos> I'm feeling stressed out because of school <bot>: ~~~ <bos>
<bos> what are the things that you hate the most <bot>: ~~~ <bos
<bos> I got my college degree, am so happy <bot>: ~~~ <bos


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<bos> i'm afraid of darkness <bot>: ive seen it before <bot>: 
<bos> Why you study <bot>: ~~~ <bos> I'm not sure if


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<bos> are you ok? <bot>: ฿฿฿฿�
<bos> are you going to see your doctor? <bot>: ฿฿�


# Prueba GPT-2 Fine Tuned

In [ ]:
tokenizer.add_special_tokens({"pad_token": "<pad>", "bos_token": "<bos>", "eos_token": "<eos>"})
tokenizer.add_tokens(["<bot>:"])
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load('/content/drive/MyDrive/PLN/PROYECTO/chatbot.pt'))
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50261, bias=False)
)

# Prueba de aplicación

In [ ]:

inpt = ''
while inpt != 'salir':
    inpt = input('Hi, how can I help? ')
    if inpt == 'test':
        print_prompt_test('/content/drive/MyDrive/PLN/PROYECTO/Prompts_prueba.txt')
        print()
    else:
      if inpt != 'salir':
        output = generate_inference(inpt)
        print(output)
      else:
        print('<bot>: Bye!')
        break

Hi, how can I help? test


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 I feel sad because my dog got lost  <bot>:   I'm sorry to hear that. 
 I am feeling good today     <bot>:   I'm so happy for you! 
 I am having a really hard time and i am feeling sad  <bot>:   I'm sorry_


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Today it's a very special day for my  <bot>:   I'm so happy for you!
 I'm feeling stressed out because of school  <bot>:   I'm sorry to hear that. 


...

 what are the things that you hate the most  <bot>:   I hate being alone_comma
 I got my college degree, am so happy  <bot>:   I am glad to hear that.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 i'm afraid of darkness  <bot>:   I'm afraid of that too_comma_ I
 Why you study  <bot>:   I do not have a college degree.  I took a college


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 are you ok?  <bot>:   I'm not quite sure.  I'm basically over it
 are you going to see your doctor?  <bot>:   I'm not sure_comma_

Hi, how can I help? salir
<bot>: Bye!
